# Entrega de modelo de Advanced Analitics

`keywords.: serving, deploy`

Este notebook tem as instruções principais para a realização da entrega de um modelo de machine learning em ambiente do AWS SageMaker.


## Setup

Seção com a preparação das variáveis do processo de deploy.

`Ref. Tempo esperado de execução: 2,15s`


In [4]:
%%time

# Célula de configurações

import os
import boto3
import re
import json
from sagemaker import get_execution_role, session

region = boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

# Você pode usar um bucket, mas certifique-se de que a permissão que escolheu para este notebook
# contempla s3:PutObject . Este é o bucket no qual os dados são capturados
bucket = session.Session(boto3.Session()).default_bucket()
print("Bucket a ser usado: {}".format(bucket))
prefix = "sagemaker/FIAP-ModelMonitor"

data_capture_prefix = "{}/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)
reports_prefix = "{}/reports".format(prefix)
s3_report_path = "s3://{}/{}".format(bucket, reports_prefix)
code_prefix = "{}/code".format(prefix)
s3_code_preprocessor_uri = "s3://{}/{}/{}".format(bucket, code_prefix, "preprocessor.py")
s3_code_postprocessor_uri = "s3://{}/{}/{}".format(bucket, code_prefix, "postprocessor.py")

print("Path de captura dos dados: {}".format(s3_capture_upload_path))
print("Path dos relatórios: {}".format(s3_report_path))
print("Path para código de pré-proc: {}".format(s3_code_preprocessor_uri))
print("Path para código de pós-proc: {}".format(s3_code_postprocessor_uri))
print("\n")

RoleArn: arn:aws:iam::325011675573:role/service-role/AmazonSageMaker-ExecutionRole-20201008T200444
Bucket a ser usado: sagemaker-us-east-2-325011675573
Path de captura dos dados: s3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/datacapture
Path dos relatórios: s3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/reports
Path para código de pré-proc: s3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/code/preprocessor.py
Path para código de pós-proc: s3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/code/postprocessor.py


CPU times: user 162 ms, sys: 31.3 ms, total: 194 ms
Wall time: 326 ms


A seguir, o arquivo `test_data/upload-test-file.txt` será enviado para o Bucket, na subpasta `test_upload/test.txt`

`Ref. Tempo esperado de execução: 0,5s`

In [6]:
%%time


boto3.Session().resource("s3").Bucket(bucket).Object("test_upload/test.txt").upload_file(
    "test_data/upload-test-file.txt"
)
print("Sucesso! Podemos seguir em frente.")

Sucesso! Podemos seguir em frente.
CPU times: user 53.1 ms, sys: 14.4 ms, total: 67.5 ms
Wall time: 122 ms


## Upload do modelo treinado

Este código carrega um modelo XGBoost pré-treinado pelo cientista de dados e que está pronto para ser implantado.  
Este modelo foi treinado usando o XGBoos. Você também pode usar seu próprio modelo pré-treinado nesta etapa.

Segundo o cientista de dados, o modelo treinado tem as seguintes características:
- Arquivo: `model/xgb-churn-prediction-model.tar.gz`
- Linguagem: `Python`
- Versão da Linguagem: `3.X`
- Framework: `xgboost`
- Versão do Framework: `0.90-1`  

`Ref. Tempo esperado de execução: 0,2s`

In [9]:
%%time

model_file = open("model/xgb-churn-prediction-model.tar.gz", "rb")
s3_key = os.path.join(prefix, "xgb-churn-prediction-model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

CPU times: user 49.8 ms, sys: 12.5 ms, total: 62.3 ms
Wall time: 132 ms


### Deploy do modelo para o Amazon SageMaker

Os códigos a seguir farãp o deploy do modelo de previsão de churn pré-treinado. 

Nesta célula, criamos o objeto `modelo` com a imagem e os dados do modelo.


`Ref. Tempo esperado de execução: 0,01s`

In [21]:
%%time

from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

model_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://{}.s3-{}.amazonaws.com/{}/xgb-churn-prediction-model.tar.gz".format(
    bucket, region, prefix
)

image_uri = retrieve("xgboost", boto3.Session().region_name, "0.90-1")

modelo = Model(image_uri=image_uri, model_data=model_url, role=role)


print("URI da imagem base: {}".format(image_uri))
print("URL do modelo: {} \n".format(model_url))

URI da imagem base: 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3
URL do modelo: https://sagemaker-us-east-2-325011675573.s3-us-east-2.amazonaws.com/sagemaker/DEMO-ModelMonitor/xgb-churn-prediction-model.tar.gz 

CPU times: user 21.4 ms, sys: 96 µs, total: 21.5 ms
Wall time: 66.5 ms


Para habilitar a captura de dados para futuramente monitorar a qualidade dos dados do modelo, especificamos a opção de captura chamada `DataCaptureConfig`.

Nós podemos capturar a carga da solicitação, a carga da resposta ou ambos com esta configuração.

A configuração de captura se aplica a todas as variantes. Vá em frente com a implantação.

`Ref. Tempo esperado de execução: 7m`

In [24]:
%%time

from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName: {}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = modelo.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)
print("\n")

EndpointName=DEMO-xgb-churn-pred-model-monitor-2021-09-16-16-37-14
-------------!CPU times: user 251 ms, sys: 25 ms, total: 276 ms
Wall time: 6min 32s


## Delete the resources

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.

In [ ]:
predictor.delete_endpoint()

In [ ]:
predictor.delete_model()